# 使用AE對電晶體進行異常檢測
注意！本程式使用cpu進行執行

## 下載資料集＆預訓練權重
該電晶體影像取自於MVTec資料集

原始資料集網址：https://www.kaggle.com/datasets/ipythonx/mvtec-ad?resource=download

In [34]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1A5FVE_dOspyOFZfSqyNMiL63XeMV6V0F' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1A5FVE_dOspyOFZfSqyNMiL63XeMV6V0F" -O data.zip && rm -rf /tmp/cookies.txt
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1gKqpGCNivTiYO6qtPrsTO-J6iRPEHlnZ' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1gKqpGCNivTiYO6qtPrsTO-J6iRPEHlnZ" -O model.pth && rm -rf /tmp/cookies.txt
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1sPig61E_1ScUjdPbjgh-M2NVQpRHzgWR' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1sPig61E_1ScUjdPbjgh-M2NVQpRHzgWR" -O model_cnn.pth && rm -rf /tmp/cookies.txt

--2023-06-27 07:25:00--  https://docs.google.com/uc?export=download&confirm=t&id=1A5FVE_dOspyOFZfSqyNMiL63XeMV6V0F
Resolving docs.google.com (docs.google.com)... 74.125.196.113, 74.125.196.139, 74.125.196.101, ...
Connecting to docs.google.com (docs.google.com)|74.125.196.113|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0o-ak-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/8a6eto8gcu7pb6evpi2kpacpm3oupjkr/1687850700000/03131947759355096649/*/1A5FVE_dOspyOFZfSqyNMiL63XeMV6V0F?e=download&uuid=403fb6b5-063f-4bee-bbdf-da79432c4df1 [following]
--2023-06-27 07:25:00--  https://doc-0o-ak-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/8a6eto8gcu7pb6evpi2kpacpm3oupjkr/1687850700000/03131947759355096649/*/1A5FVE_dOspyOFZfSqyNMiL63XeMV6V0F?e=download&uuid=403fb6b5-063f-4bee-bbdf-da79432c4df1
Resolving doc-0o-ak-docs.googleusercontent.com (doc-0o-ak-docs.googleusercontent.com)... 172.217.203.132,

In [ ]:
!tar xvf data.zip

## 宣告須使用的套件庫

In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
import numpy as np
import math
import cv2
from google.colab.patches import cv2_imshow

## 定義AE(此範例中使用linear函數)

In [ ]:
class Autoencoder(nn.Module):
    def __init__(self, input_dim):
        super(Autoencoder, self).__init__()

        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 512),
            nn.Linear(512, 512),
            nn.Linear(512, 256),
            nn.Linear(256, 128),
            nn.ReLU()
        )

        self.decoder = nn.Sequential(
            nn.Linear(128, 256),
            nn.Linear(256, 512),
            nn.Linear(512, 512),
            nn.Linear(512, input_dim),
            nn.Sigmoid()
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

## 補充： CNN版本


In [ ]:
class Autoencoder(nn.Module):
    def __init__(self, input_dim):
        super(Autoencoder, self).__init__()

        self.encoder = nn.Sequential(
            nn.Conv2d(3, 8,3,padding='same'),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(8,32,3,padding='same'),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(32,32,3,padding='same'),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(32,32,3,padding='same'),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(32,64,3,padding='same'),
            nn.ReLU(),
        )

        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(64,32,3,padding=1),
            nn.ReLU(),
            nn.Upsample(scale_factor=2,mode='bilinear'),
            nn.ConvTranspose2d(32,32,3,padding=1),
            nn.ReLU(),
            nn.Upsample(scale_factor=2,mode='bilinear'),
            nn.ConvTranspose2d(32,32,3,padding=1),
            nn.ReLU(),
            nn.Upsample(scale_factor=2,mode='bilinear'),
            nn.ConvTranspose2d(32,8,3,padding=1),
            nn.ReLU(),
            nn.Upsample(scale_factor=2,mode='bilinear'),
            nn.ConvTranspose2d(8,3,3,padding=1),
            nn.Sigmoid()
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

## 製作訓練與測試資料集

In [36]:
# 設置資料路徑
data_path = './data/transistor/'

# 定義預處理轉換
transform = transforms.Compose([
    transforms.Resize((256, 256)),   # 調整圖像大小
    transforms.ToTensor()          # 轉換為張量
])

# 加載訓練集
train_dataset = ImageFolder(root=data_path+'/train', transform=transform)

# 加載測試集
test_dataset = ImageFolder(root=data_path+'/test', transform=transform)

# 創建訓練集和測試集的數據加載器
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1, shuffle=False)


## 定義訓練參數
**如果有要進行訓練，最下面的train請設定為1,否則為0**

為加快demo，此範例使用預先訓練的模型進行

In [51]:
# 定義模型超參數
input_dim = 256 * 256 * 3   # 輸入維度 (64x64x3)

# 創建自編碼器模型
model = Autoencoder(input_dim)

# 定義損失函數和優化器
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)
# 定義訓練步數
num_epochs = 500
# 是否進行訓練或是使用預訓練權重進行預測
train = False
cnn = False

In [52]:
if train:

    for epoch in range(num_epochs):
        for data in train_loader:
            images, _ = data
            if not cnn:
                images = images.view(images.size(0), -1)

            # 正向傳播
            outputs = model(images)
            loss = criterion(outputs, images)

            # 反向傳播和優化
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        # 每個epoch打印訓練損失
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')
    torch.save(model,"./model.pth")
#-------------------------------
else:
    if cnn:
        model = torch.load("./model_cnn.pth")
    else:
        model = torch.load("./model.pth")

In [55]:
#定義：正常為正樣本，異常為負樣本
#宣告判定正確（TP,TN）、異常誤判(FP)、正常過殺(FN)計數器
TP,TN,FP,FN = 0,0,0,0
#定義異常閥值
if not cnn:
    th = 0.0024
else:
    th = 0.0013
#不計算梯度
with torch.no_grad():

    for i, (images, label) in enumerate(test_loader):
        if not cnn:
            images = images.view(images.size(0), -1)
        outputs = model(images)
        loss = criterion(outputs, images)
        #輸出結果轉換成圖像
        if not cnn:
            images = images.view(1,3,256,256)
            outputs = outputs.view(1,3,256,256)
        images = np.transpose(images[0].numpy(), (1, 2, 0))
        outputs = np.transpose(outputs[0].numpy(), (1, 2, 0))
        images = cv2.cvtColor(np.asarray(images*255,dtype=np.uint8), cv2.COLOR_RGB2GRAY)
        outputs = cv2.cvtColor(np.asarray(outputs*255,dtype=np.uint8), cv2.COLOR_RGB2GRAY)
        #計算輸入與輸出的差異並製作成熱圖
        diff = cv2.absdiff(images,outputs)
        diff = cv2.applyColorMap(diff, cv2.COLORMAP_JET)
        cv2_imshow(cv2.resize(images,(400,400)))
        cv2_imshow(cv2.resize(outputs,(400,400)))
        cv2_imshow(cv2.resize(diff,(400,400)))
        #根據loss閥值判定正常/異常
        if loss>th:
            if label == 4:
                FN+=1
            else:
                TN+=1
        else:
            if label == 4:
                TP+=1
            else:
                FP+=1

    print("成功判定為正常數：",TP)
    print("成功判定為異常數：",TN)
    print("異常漏判數：",FP)
    print("正常過殺數：",FN)


成功判定為正常數： 43
成功判定為異常數： 28
異常漏判數： 12
正常過殺數： 17
